# Списки

Списки являются простыми структурами данных, которые удобны для некоторых задач. По сути своей они представляют просто набор элемнтов, где каждый хранит значение и указатель на следующий элемент. Последний элемент хранит указатель на несуществующий элемент (NULL).

In [1]:
#include<vector>
#include<iostream>
#include <functional>

In [2]:
class Node { // объект связанного списка
public:
    Node() : next(nullptr) {} 
    Node(int v) : value(v), next(nullptr) {}
    ~Node() {}
    
    Node * next; // Указатель на следующий 
    int value; // значение
};

In [3]:
class List{
public:
    List() : begin(nullptr) {}
    ~List() {
        Node* current = begin;
        Node* toDel;
        while(current != nullptr) { // удаляем все элементы
            toDel = current;
            current = current->next;
            delete toDel;
        }
    }
    
    void Insert(int value) { // вставка в список
        Node * newNode = new Node(value); // создаем новый элемент
        newNode->next = begin; // вставляем его в самое начало
        begin = newNode; // заменяя begin на него
    }
    
    void for_each(std::function<void(int)> f) { // конструкция для удобного итерирования по коллекции
        Node* current = begin;
        while(current != nullptr) {
            f(current->value);
            current = current->next;
        }
    }
    
    Node * begin;
};

In [4]:
List l1;
for(int i = 0; i < 10; i++) { // заполняем 
    l1.Insert(i*i);
}
l1.for_each([](int e){ // выводим
    std::cout << e << ' ';
});
std::cout << std::endl;

81 64 49 36 25 16 9 4 1 0 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f778d3256e0


Как можно видеть, все числа были выведены в обратном порядке, так как мы всегда вставляли число в начало.

# Обращение списка

Предположим, что теперь мы хотим иметь прямой порядок, а не обратный после вставки элементов. Для этого требуется развернуть список. Данный алгоритм крайне прост - мы идем по списку и меняет значения next у элементов на предыдущий.

In [5]:
void reverse_list(List &l) {
    Node* prev = nullptr; // предыдущий элемент. Изначально пустой
    Node* current = l.begin; // текущий
    while(current != nullptr) {
        Node * next = current->next;
        current->next = prev; // меняем на предыдущий элемент
        prev = current; // сдвигаемся по списку
        current = next;
    }
    l.begin = prev;
}

In [6]:
reverse_list(l1);
l1.for_each([](int a) {
    std::cout << a << ' ';
});
std::cout << std::endl;

0 1 4 9 16 25 36 49 64 81 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f778d3256e0


Однако данный подход в лоб не является оптимальным, так как для обращения списка из n элементом, нам необходимо будет сделать n действий.

Для того, чтобы решить данную проблему, можно использовать двусвязный список - это такой же список из элементов, однако теперь каждый элемент помимо предыдущего содержит указатель на предыдущий элемент. Также в списке теперь храниться указатель на последний элемент. 

In [7]:
class DoubleNode {
public:
    DoubleNode() : right(nullptr), left(nullptr) {}
    DoubleNode(int v) : value(v), right(nullptr), left(nullptr) {}
    ~DoubleNode(){}
    
    int value;
    DoubleNode* right;
    DoubleNode* left;
};

In [8]:
class DoubleList {
public:
    DoubleList() : begin(nullptr), end(nullptr), right(true) {}
    ~DoubleList() {
        DoubleNode* current = begin;
        DoubleNode* toDel;
        while(current != nullptr) { // удаляем все элементы
            toDel = current;
            current = right ? current->right : current->left;
            delete toDel;
        }
    }
    
    void Insert(int value) {
        DoubleNode* e = new DoubleNode(value);
        
        if(begin == nullptr) end = e;
        
        if(right) e->right = begin;
        else e->left = begin;
        
        if(begin != nullptr) {
            if(right) begin->left = e;
            else begin->right = e;
        }
        
        begin = e;
    }
    
    void for_each(std::function<void(int)> f) { // конструкция для удобного итерирования по коллекции
        DoubleNode* current = begin;
        while(current != nullptr) {
            f(current->value);
            if(right) current = current->right;
            else current = current->left;
        }
    }
    
    DoubleNode * begin;
    DoubleNode * end;
    bool right; // указатель на то, в какую сторону двигаться
};

In [9]:
DoubleList l2;
for(int i = 0; i < 15; i++) {
    l2.Insert(i*i*i);
}
l2.for_each([](int i) {
    std::cout << i << ' ';
});
std::cout << std::endl;

2744 2197 1728 1331 1000 729 512 343 216 125 64 27 8 1 0 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f778d3256e0


Теперь для обращения списка необходимо лишь поменять местами начало и конец, а также сделать пометку, что движение по списку теперь ведется в другом направлении.

In [10]:
void reverse_double_list(DoubleList &l) {
    std::swap(l.begin, l.end); // меняем начало и конец
    l.right = !l.right; // меняем направление движения
}

In [11]:
reverse_double_list(l2);
l2.for_each([](int i) {
    std::cout << i << ' ';
});
std::cout << std::endl;

0 1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f778d3256e0


Теперь мы можем очень быстро обращать списки, а также очень удобно перемещаться по ним в обе стороны.

# Обнаружение циклов

Часто бывает, что подобные списковые структуры имею внутри себя цикл - то есть последний элемент ссылается на какой-то предыдущий. Иногда это бывает полезно, однако это может вызвать определенные проблемы с обработкой подобных структур, поэтому полезно уметь определять циклы в списках.

Простой алгоритм обнаружения циклов базируется на алгоритме обращения списка, который мы описали в начале лекции: если мы начнем разворачивать такой список, то мы сначала дойдем до конца списка, вклинимся в середину списка и так как все эти элементы мы уже развернули, то мы пойдем обратно к началу.
Таким образом в конце мы дойдем до начала списка и вновь обернем первый элемент.

В итоге, для того чтобы определить наличие цикла в списке, необходимо разрернуть его и потом посмотреть на первый элемент. Если он ссылается на пустой элемент, то значит обращение произошло корректно и цикла в списке нет, однако если он указывает на какой-то реальный элемент, это означает, что мы развернули его дважды и в списке присутствует цикл.

In [12]:
List l3;
Node* middle;
Node* end;
for(int i = 0; i < 10; i++) { // создаем ситуацию цикла
    l3.Insert(i);
    if(i == 0) end = l3.begin;
    if(i == 5) middle = l3.begin;
}

end->next = middle; // замыкаем список

(Node *) 0x3a0f1f0


In [13]:
Node * cur = l3.begin;
int count = 0;
while(cur != nullptr and count < 50) { // что плохого может произойти - при последовательной обработке мы можем
    std::cout << cur->value << ' '; // началь бесконено долго
    cur = cur->next;
    count++;
}

9 8 7 6 5 4 3 2 1 0 5 4 3 2 1 0 5 4 3 2 1 0 5 4 3 2 1 0 5 4 3 2 1 0 5 4 3 2 1 0 5 4 3 2 1 0 5 4 3 2 

In [14]:
bool find_cycle(List &l) {
    Node* prev = nullptr; // предыдущий элемент. Изначально пустой
    Node* current = l.begin; // текущий
    while(current != nullptr) {
        Node * next = current->next;
        current->next = prev; // меняем на предыдущий элемент
        prev = current; // сдвигаемся по списку
        current = next;
    }
    if(l.begin->next == nullptr) return false; // если развернули только один раз
    else return true; // если несколько
}

In [15]:
std::cout << find_cycle(l1) << std::endl;

0


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f778d3256e0


In [16]:
std::cout << find_cycle(l3) << std::endl;

1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f778d3256e0


# Обнаружение методом двух указателей

Существенный минус рассмотренного метода - исходный список становится измененым и более непригодным к использованию.

Для разрешения этой проблемы можно использовать следующий алгоритм:
* Создаем два указателя на начало
* Далее двигаемся ими по списку, однако с разной скоростью - первый за одну итерацию проходит один элемент, а второй два элемента
* Если оба указателя в итоге дойдут до конца - значит, что список корректный и цикла нет
* Если эти два указателя встретятся, то есть будут указывать на один и тот же элемент, то это будет означать, что они оба попали в цикл и более быстрый догнал более медленный

In [17]:
bool find_cycle2(List &l) {
    Node* a = l.begin; // два указателя
    Node* b = l.begin;
    for(;;) {
        a = a->next; // двигаемся первым 
        if(a == b) return true;
        if(a == nullptr) return false;
        a = a->next; // два раза
        if(a == b) return true;
        if(a == nullptr) return false;
        b = b->next; // и один раз вторым
        if(a == b) return true; // проверяем условия после каждого движения
        if(b == nullptr) return false;
    }
}

In [18]:
List a1, a2;
for(int i = 0; i < 10; i++) {
    a1.Insert(i);
    a2.Insert(i);
    if(i == 0) end = a2.begin;
    if(i == 5) middle = a2.begin;
}
end->next = middle;

(Node *) 0x3a87460


In [19]:
std::cout << find_cycle2(a1) << std::endl;

0


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f778d3256e0


In [20]:
std::cout << find_cycle2(a2) << std::endl;

1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f778d3256e0
